In [8]:
import pandas as pd

# Założenia finansowe 
S_LIFE = 100000  # Suma ubezpieczenia na życie
S_ENDOW = 10000  # Suma ubezpieczenia na dożycie
i = 0.0238       # Stopa techniczna KNF na 2023 r.
v = 1 / (1 + i)
n = 10

# Wczytanie oczyszczonych danych
df = pd.read_csv('processed_mortality.csv')

a_x_n_life = 0    # Wartość ubezpieczenia na życie
k_px = 1.0        # Prawdopodobieństwo przeżycia k lat (0_px = 1)
suma_renty = 0    # Do obliczenia renty życiowej (ä_x:n) 

print(f"--- Obliczenia dla mezczyzny w wieku {int(df.iloc[0]['Age'])} lat ---")

for k in range(n):
    qx_plus_k = df.iloc[k]['qx']
    
    # 1. Obliczenie składnika ubezpieczenia na życie 
    term_life = (v**(k+1)) * k_px * qx_plus_k
    a_x_n_life += term_life
    
    # 2. Obliczenie składnika renty (potrzebnej do składki rocznej)
    suma_renty += (v**k) * k_px
    
    # Aktualizacja k_px na kolejny rok
    k_px *= (1 - qx_plus_k)

# 3. Ubezpieczenie na dożycie (po 10 latach) 
a_x_n_endow = (v**n) * k_px

# OBLICZENIE SKŁADEK JEDNORAZOWYCH (NSP)
nsp_life = a_x_n_life * S_LIFE
nsp_endow = a_x_n_endow * S_ENDOW
nsp_total = nsp_life + nsp_endow

# OBLICZENIE ROCZNEJ SKŁADKI NETTO 
# P = (NSP na życie + NSP na dożycie) / renta
annual_premium = nsp_total / suma_renty

print("\n--- PODSUMOWANIE PROJEKTU ---")
print(f"NSP za ubezpieczenie na życie (100 tys.): {nsp_life:.2f} zł")
print(f"NSP za ubezpieczenie na dożycie (10 tys.): {nsp_endow:.2f} zł")
print(f"Łączna jednorazowa składka netto: {nsp_total:.2f} zł")
print(f"Wartość renty życiowej n-letniej (ä_x:n): {suma_renty:.6f}")
print(f"--- ROCZNA SKŁADKA NETTO: {annual_premium:.2f} zł ---")

# WERYFIKACJA JEDYNKI AKTUARIALNEJ 
d = 1 - v
# A_x_n to suma wartości aktuarialnych dla S=1
A_x_n_unit = a_x_n_life + a_x_n_endow 

jedynka_aktuarialna = d * suma_renty + A_x_n_unit

print("\n--- TEST POPRAWNOŚCI (JEDYNKA AKTUARIALNA) ---")
print(f"LHS (lewa strona równania): {jedynka_aktuarialna:.4f}")
if round(jedynka_aktuarialna, 8) == 1.0:
    print("STATUS: Formuły są poprawne (wynik = 1).")
else:
    print("STATUS: Błąd w formułach.")

--- Obliczenia dla mezczyzny w wieku 30 lat ---

--- PODSUMOWANIE PROJEKTU ---
NSP za ubezpieczenie na życie (100 tys.): 636.73 zł
NSP za ubezpieczenie na dożycie (10 tys.): 7846.29 zł
Łączna jednorazowa składka netto: 8483.02 zł
Wartość renty życiowej n-letniej (ä_x:n): 8.990686
--- ROCZNA SKŁADKA NETTO: 943.53 zł ---

--- TEST POPRAWNOŚCI (JEDYNKA AKTUARIALNA) ---
LHS (lewa strona równania): 1.0000
STATUS: Formuły są poprawne (wynik = 1).
